In [386]:
import pandas as pd
import numpy as np
from csv import reader
from numpy import linalg as la
from sys import exit
from haversine import haversine
import math
from operator import itemgetter
from numpy import dot
from numpy.linalg import norm
#시각화 도구
import folium
from folium.features import DivIcon
import random
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [387]:
# 전역 변수 설정
FILE_SAVE_DIR = './Data_set_0826/' #있다면 추가 해주기 (상대경로)
#Call CSV file
def load_GPS_set_to_array(UID): # Data load
    try:
        data_set = pd.read_csv(FILE_SAVE_DIR + UID +"_gps"+".csv", sep=",", dtype='unicode')
        data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime'])
        data_set['TIME STAMP'] = pd.to_datetime(data_set['TIME STAMP']).map(pd.Timestamp.timestamp) # to unix time 
        re_data_set = data_set[["TIME STAMP","uid","latitude","longitude","provider","speed"]]
        return re_data_set
    except FileNotFoundError as e:
        pass
    
def extract_raw_data(UID):
    Dataframe = load_GPS_set_to_array(UID)
    marker_list = []
    for count in range(len(Dataframe.index)):
        marker_point_lati = float(Dataframe.loc[count,'latitude'])
        marker_point_long = float(Dataframe.loc[count,'longitude'])
        marker_point_speed = float(Dataframe.loc[count,'speed'])
        marker_point_time = Dataframe.loc[count,'TIME STAMP']
        marker_point = [marker_point_time, marker_point_speed, marker_point_lati, marker_point_long]
        marker_list.append(marker_point)
    return marker_list

In [388]:
joongho_raw = extract_raw_data('9E7F4CB4F4A6A6D4')

In [389]:
def set_state(accuracy):
    variance = accuracy * accuracy
    return variance

def export_new_point(result_map, speed, longitude, latitude, timestamp):
    result_map.append([timestamp, speed, latitude, longitude])

def kalman_filter(variance, new_speed, latitude, new_latitude, longitude, new_longitude, timestamp, new_timestamp, new_accuracy):

    duration = new_timestamp - timestamp
    if (duration > 0) :
        # time has moved on, so the uncertainty in the current position increases
        variance += duration * new_speed * new_speed / 1000

    # Kalman gain matrix 'k' = Covariance * Inverse(Covariance + MeasurementVariance)
    # because 'k' is dimensionless,
    # it doesn't matter that variance has different units to latitude and longitude
    k = variance / (variance + new_accuracy * new_accuracy)
    # apply 'k'
    latitude += k * (new_latitude - latitude)
    longitude += k * (new_longitude - longitude)
    # new Covariance matrix is (IdentityMatrix - k) * Covariance
    variance = (1 - k) * variance

    return new_speed, longitude, latitude, duration
        

In [390]:
# raw_list = [timestamp(unix), speed, lati, long]
FILTER_CONSTANT = 300
def precess(raw_list):
    variance = -1
    Result_map = []
    for idx in range(len(raw_list)):
        if (variance < 0) :
            variance = set_state(FILTER_CONSTANT)
        else :
            new_speed, longitude, latitude, duration = kalman_filter(variance, raw_list[idx][1], raw_list[idx-1][2], raw_list[idx][2], 
                                                                     raw_list[idx-1][3], raw_list[idx][3], raw_list[idx-1][0],raw_list[idx][0],FILTER_CONSTANT)
                # Export new point
            export_new_point(Result_map, new_speed, longitude, latitude, duration)
    return Result_map
filter_list = precess(joongho_raw)

In [391]:
kalman_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사

def draw_Polyline_map(map, marker_list, draw_color):
    replace_list = []
    for marker_point in marker_list:
        replace_list.append(marker_point[2:4])
    folium.PolyLine(locations=replace_list,tooltip='Polyline',color=draw_color).add_to(map)
    
def draw_circle_map(map, marker_list,draw_radius,draw_color):
    for marker_point in marker_list:
        folium.CircleMarker(location=marker_point[2:4], radius=draw_radius, color= draw_color).add_to(map)


In [392]:
#contact result
kyounghwan = '3963650B38CAF112'
hyeryeong = '4CCF4C0B0F953026'
pyeongju = '8A57AE95DD4629A7'
joongho = '9E7F4CB4F4A6A6D4'
yujin = 'F9888CC2D3053565'

kyounghwan_raw = extract_raw_data('3963650B38CAF112')[0:31]
hyeryeong_raw = extract_raw_data('4CCF4C0B0F953026')[0:31]
pyeongju_raw = extract_raw_data('8A57AE95DD4629A7')[0:31]
joongho_raw = extract_raw_data('9E7F4CB4F4A6A6D4')[0:31]
yujin_raw = extract_raw_data('F9888CC2D3053565')[0:31]

kyounghwan_filter = precess(kyounghwan_raw)[0:31]
hyeryeong_filter = precess(hyeryeong_raw)[0:31]
pyeongju_filter = precess(pyeongju_raw)[0:31]
joongho_filter = precess(joongho_raw)[0:31]
yujin_filter = precess(yujin_raw)[0:31]

draw_Polyline_map(kalman_map,hyeryeong_filter,"blue")
draw_circle_map(kalman_map,hyeryeong_filter,3,"blue")
draw_Polyline_map(kalman_map,hyeryeong_raw,"red")
draw_circle_map(kalman_map,hyeryeong_raw,3,"red")
# kalman_map

In [393]:
member_list = ['3963650B38CAF112','4CCF4C0B0F953026','8A57AE95DD4629A7','9E7F4CB4F4A6A6D4','F9888CC2D3053565']
DISTANCE_TH = 30
COSINESIM_TH = 0.7
RESULT_LIST = []

for patient_id in member_list:
    patient_raw = extract_raw_data(patient_id)
    for tester_id in member_list:
        if tester_id != patient_id:
            result_map = {}
            result_map["patient"] = patient_id
            result_map["tester"] = tester_id
            tester_raw = extract_raw_data(tester_id)
            
            patient_filter = precess(patient_raw)
            tester_filter = precess(tester_raw)
            
            predict_list = []
            for idx in range(len(patient_filter)):
                try:
                    distance = haversine(patient_filter[idx][2:4],tester_filter[idx][2:4], unit = 'm')
                    # print("patient : " + patient_id + "/ tester : " + tester_id + " >>>> distance : " + str(distance))
                    if distance <= DISTANCE_TH:
                        # print("patient : " + patient_id + "/ tester : " + tester_id + " >>>> distance : " + str(distance))
                        patient_speed_x = patient_filter[idx + 1][2] - patient_filter[idx][2]
                        patient_speed_y = patient_filter[idx + 1][3] - patient_filter[idx][3]
                        tester_speed_x = tester_filter[idx + 1][2] - tester_filter[idx][2]
                        tester_speed_y = tester_filter[idx + 1][3] - tester_filter[idx][3]
                        cosine_sim = dot([patient_speed_x,patient_speed_y], [tester_speed_x,tester_speed_y]) / (norm([patient_speed_x,patient_speed_y]) * norm([tester_speed_x,tester_speed_y]))
                        if cosine_sim >= COSINESIM_TH:
                        # if cosine_sim >= COSINESIM_TH or cosine_sim <= -COSINESIM_TH:
                            # print("<<<<<< cosine : " + str(cosine_sim))
                            predict_list.append(1)
                        else:
                            predict_list.append(0)
                    else:
                        predict_list.append(0)
                except IndexError:
                    break
            result_map["predict"] = predict_list
            RESULT_LIST.append(result_map)
# print(RESULT_LIST)

C:\Users\User\AppData\Local\Temp/ipykernel_7120/3779354326.py:29: RuntimeWarning: invalid value encountered in double_scalars
  cosine_sim = dot([patient_speed_x,patient_speed_y], [tester_speed_x,tester_speed_y]) / (norm([patient_speed_x,patient_speed_y]) * norm([tester_speed_x,tester_speed_y]))


In [394]:
def get_groundtruth(patient_id, tester_id):
    try:
        data_set = pd.read_csv(FILE_SAVE_DIR + patient_id +"_gps"+".csv", sep=",", dtype='unicode')
        data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime'])
        data_set['TIME STAMP'] = pd.to_datetime(data_set['TIME STAMP']).map(pd.Timestamp.timestamp) # to unix time 
        re_data_set = data_set[["c_"+tester_id]]
        return re_data_set
    except FileNotFoundError as e:
        pass

def get_result():
    RESULT_MAP = {}
    for result in RESULT_LIST:
        groundtruth = get_groundtruth(result["patient"], result["tester"]).values.tolist()
        gt = []
        for e in groundtruth:
            gt.append(int(e[0]))
        predict_res = classification_report(gt[0:len(result["predict"])] ,result["predict"])
        print("patient : "+result["patient"])
        print("tester : "+result["tester"])
        print("get : "+str(gt[0:len(result["predict"])]))
        print("pred : "+str(result["predict"]))
        print(predict_res)
        RESULT_MAP["patient"] = result["patient"]
        RESULT_MAP["tester"] = result["tester"]
        RESULT_MAP["result"] = predict_res
    return RESULT_MAP


def total_result():
    total_gt = []
    total_pre = []
    for result in RESULT_LIST:
        groundtruth = get_groundtruth(result["patient"], result["tester"]).values.tolist()
        gt = []
        for e in groundtruth:
            gt.append(int(e[0]))
        total_gt.append(gt[0:len(result["predict"])])
        total_pre.append(result["predict"])
    total_gt = sum(total_gt, [])
    total_pre = sum(total_pre, [])
    predict_res = classification_report(total_gt ,total_pre)
    print(predict_res)
    return predict_res
# get_result()
total_result()

              precision    recall  f1-score   support

           0       0.78      0.99      0.87       907
           1       0.97      0.39      0.56       431

    accuracy                           0.80      1338
   macro avg       0.87      0.69      0.72      1338
weighted avg       0.84      0.80      0.77      1338



'              precision    recall  f1-score   support\n\n           0       0.78      0.99      0.87       907\n           1       0.97      0.39      0.56       431\n\n    accuracy                           0.80      1338\n   macro avg       0.87      0.69      0.72      1338\nweighted avg       0.84      0.80      0.77      1338\n'